## <span style="color:purple">Imports, masks, conversions

In [1]:
import xarray as xr
import os
import sys
# Custom utilities
import utils
from utils.load_variable import load_variables_for_all_years
import pandas as pd

NPAC = xr.open_dataarray('/glade/u/home/jzweifel/jupyter_workspaces/Masters_paper_analysis/weighting_and_masks/NPO_binary_map_lower_bound_20_deg.nc')
East_Asia = xr.open_dataarray('/glade/u/home/jzweifel/jupyter_workspaces/Masters_paper_analysis/weighting_and_masks/CJK_binary_assign_coords.nc')
area = xr.open_dataarray('/glade/u/home/jzweifel/jupyter_workspaces/Masters_paper_analysis/weighting_and_masks/AREA.nc')

second = 1
minute = 60 * second
hour = 60 * minute
day = 24 * hour
seconds_per_year = 365 * day

kg = 1
Tg_per_kg = 1e-9 * kg


## <span style="color:purple"> Budget variables

In [2]:
emission_vars = ['SFso4_a1', 'SFso4_a2', 'SFso4_a3', 'so4_a1_CMXF', 'so4_a2_CMXF']
# surface fluxes & vertically integrated external forcing (kg/m2/s)

aq_prod_vars = ['so4_c1AQSO4', 'so4_c1AQH2SO4', 'so4_c2AQSO4', 'so4_c2AQH2SO4', 'so4_c3AQSO4', 'so4_c3AQH2SO4']
# aqueous phase chemistry (kg/m2/s)

dry_chem_nuc_vars = ['so4_a2_sfnnuc1']
# new particle nucleation column tendency (kg/m2/s)

dry_chem_cond_vars = ['so4_a1_sfgaex1', 'so4_a2_sfgaex1', 'so4_a3_sfgaex1']
# gas-aerosol-exchange primary column tendency (kg/m2/s)

dry_dep_variables = ['so4_a1DDF', 'so4_a2DDF', 'so4_a3DDF', 'so4_c1DDF', 'so4_c2DDF', 'so4_c3DDF']
# dry deposition flux at bottom (grav + turb) (kg/m2/s)

wet_dep_variables = ['so4_a1SFWET', 'so4_a2SFWET', 'so4_a3SFWET', 'so4_c1SFWET', 'so4_c2SFWET', 'so4_c3SFWET']
# wet deposition flux at surface (kg/m2/s)

## <span style="color:purple">Function to compute regional budget change across time between simulations</span>

In [3]:
def compute_budget_change(input_season, data_dict, region_mask, year1, year0, area, seconds_per_year, Tg_per_kg, flip_sign=False):
    
    # difference
    diff = data_dict[year1].where(region_mask) - data_dict[year0].where(region_mask)
    
    # sign convention
    if flip_sign:
        diff = diff * -1
    
    # scale and reduce
    result = (diff * area * seconds_per_year * Tg_per_kg).groupby("time.season").mean(dim=['time']).sel(season=input_season).sum(dim=['lat', 'lon']).to_array().sum()
    
    return result

In [4]:
import pandas as pd

data_groups = {
    "emission": load_variables_for_all_years('h1', emission_vars),
    "aq_prod": load_variables_for_all_years('h1', aq_prod_vars),
    "dry_chem_nuc": load_variables_for_all_years('h1', dry_chem_nuc_vars),
    "dry_chem_cond": load_variables_for_all_years('h1', dry_chem_cond_vars),
    "dry_dep": load_variables_for_all_years('h1', dry_dep_variables),
    "wet_dep": load_variables_for_all_years('h1', wet_dep_variables),
}

regions = {
    "NPAC": NPAC,
    "East_Asia": East_Asia,
}

years = ("2006", "1970")

budgets = {}

for category, dataset in data_groups.items():
    budgets[category] = {}
    for region_name, mask in regions.items():
        flip = (category == "wet_dep")
        result = compute_budget_change('JJA', dataset, mask, *years, area, seconds_per_year, Tg_per_kg, flip_sign=flip).item()
        budgets[category][region_name] = result   # <--- missing assignment

df_JJA = pd.DataFrame(budgets)

budgets = {}

for category, dataset in data_groups.items():
    budgets[category] = {}
    for region_name, mask in regions.items():
        flip = (category == "wet_dep")
        result = compute_budget_change('DJF', dataset, mask, *years, area, seconds_per_year, Tg_per_kg, flip_sign=flip).item()
        budgets[category][region_name] = result   # <--- missing assignment

df_DJF = pd.DataFrame(budgets)

## <span style="color:purple">Calculating Transportation as Residual:

In [25]:
# JJA transport
df_JJA["transport"] = (
    (df_JJA["dry_dep"] + df_JJA["wet_dep"])   # ΔD
    - df_JJA["emission"]                      # - ΔE
    - df_JJA["aq_prod"]                       # - ΔPaq
    - (df_JJA["dry_chem_cond"] + df_JJA["dry_chem_nuc"])  # - ΔPgas
)


# DJF transport
df_DJF["transport"] = (
    (df_DJF["dry_dep"] + df_DJF["wet_dep"])
    - df_DJF["emission"]
    - df_DJF["aq_prod"]
    - (df_DJF["dry_chem_cond"] + df_DJF["dry_chem_nuc"])
)

In [30]:
df_transport = pd.concat(
    {"JJA": df_JJA["transport"], "DJF": df_DJF["transport"]},
    axis=1
)
df_transport["JJA:DJF"] = df_transport["JJA"] / df_transport["DJF"]
df_transport

,JJA,DJF,JJA:DJF
NPAC,1.393989,1.038175,1.342731
East_Asia,-4.413415,-3.194614,1.381518


In [32]:
df_JJA

,emission,aq_prod,dry_chem_nuc,dry_chem_cond,dry_dep,wet_dep,transport
NPAC,0.000007,0.489425,0.000577,0.427820,0.179695,2.132123,1.393989
East_Asia,1.228960,5.995993,0.005246,10.046938,1.689651,11.174072,-4.413415


In [33]:
df_DJF

,emission,aq_prod,dry_chem_nuc,dry_chem_cond,dry_dep,wet_dep,transport
NPAC,-0.000030,1.615273,0.000521,0.216425,0.225552,2.644812,1.038175
East_Asia,1.346197,4.316201,0.003132,4.261006,1.752264,4.979659,-3.194614
